# How I parse & validate CSVs with FSharp

## Introduction

I use F# for common everyday tasks - like parsing/validating CSVs, XMLs, and JSONs, and making http api requests. In my day job I wrangle a lot of data - moving data from one location, transforming it into various formats, and loading it into another. F# is great for this - it has an expressive yet concise syntax similar to Ruby/Python, built-in types like `Option` and `Result`, and unique language features like `Active Patterns` and to parse and validate data. I also make heavy use of its discriminated unions, records, and units of measure to model that data as well as computation expressions to help compose those parsers/validators together. F# has a solid feature set that clicks with me and I have yet to experience in any other language.

This article is about a basic, everyday task: parsing and validating CSVs with F#. This means reading CSVs from disk with the `FSharp.Data` library, making extensive use of F#'s `Result` type to validate CSV fields, active patterns and computation expressions to compose results together, and modeling the application's domain using F#'s algebraic data types.

## An example CSV: Patient Vital Statuses

I currently work as a software engineer at the James Cancer Hospital & Solove Research Institute at the Ohio State University Medical Center. One of the data sources I read in with F# are patient's vital statuses.

### Modeling the Domain

A patient's vital status can be one of three options: they are either alive, dead, or lost to followup. If they're dead, they have a year of death, an age of death, as well as a cause of death. F# is excellent at succinctly modeling this domain:

In [ ]:
type VitalStatus =
    | Alive
    | Dead of DeathInfo
    | ``Lost to follow-up``
and DeathInfo = {
    YearOfDeath: YearOfDeath
    AgeAtDeath: AgeAtDeath
    CauseOfDeath: CauseOfDeath
}
and YearOfDeath = YearOfDeath of year : uint
and AgeAtDeath = AgeAtDeath of age : uint
and CauseOfDeath =
    | ``Due to Cancer``
    | ``Due to Other Cause``
    | ``Probably Due to Cancer``
    | ``Unknown Cause``
    | ``Unlikely Due to Cancer``

type PatientVitalStatus = {
    PatientId: PatientId
    VitalStatus: VitalStatus
}
and PatientId = PatientId of id: string

Each instance of `PatientVitalStatus` must have a patient id and a vital status. That vital status can either be `Alive` or `Dead`. If the `VitalStatus` is `Dead`, there must be `DeathInfo` associated with it: the `YearOfDeath`, `AgeAtDeath`, and `CauseOfDeath`. All *valid* rows in the "vital statuses" csv will be turned into instances of the `PatientVitalStatus` type. There is no way to create this type that is not a valid instance of a patient's vital status.

I like to use "single case discriminated unions types" for the `YearOfDeath` and `AgeAtDeath` to signify to other code readers that instead of just everyday, unremmarkable unsigned integers, they instead have semantic, distinct meanings. If you'd like to read more about single cased union types and the rationale behind using them, the venerable Scott Wlaschin has an excellent blog post on them, [here](https://fsharpforfunandprofit.com/posts/designing-with-types-single-case-dus/).

Getting back to validating csvs with F#, below are two instances of valid patient statuses. Note that it is *impossible* to create an invalid vital status.

In [ ]:
let deadPatient = {
    PatientId = PatientId "A000001"
    VitalStatus = Dead {
        YearOfDeath = YearOfDeath 2015u
        AgeAtDeath = AgeAtDeath 65u
        CauseOfDeath = ``Due to Cancer``
    }
}

let alivePatient = {
    PatientId = PatientId "A000002"
    VitalStatus = Alive
}

let barelyDeadPatient = {
    PatientId = PatientId "B000001"
    VitalStatus = ``Lost to follow-up`` }

[ deadPatient; alivePatient; barelyDeadPatient ]

index PatientId VitalStatus 0 id A000001 Item { { YearOfDeath = YearOfDeath 2015u
 AgeAtDeath = AgeAtDeath 65u
 CauseOfDeath = Due to Cancer }: YearOfDeath: { YearOfDeath 2015u: year: 2015 }, AgeAtDeath: { AgeAtDeath 65u: age: 65 }, CauseOfDeath: Due to Cancer } 1 id A000002 Alive 2 id B000001 Lost to follow-up

## Validating Inputs

Each row in the vital statuses csv represents an input to be parsed and validated. If the row is valid, it'll be turned into patient vital status. If the row is invalid, a validation error for the row will be returned.

Below are examples of rows found within the csv. The first few rows show examples of valid patient vital statuses. The last few rows show examples of invalid patient vital statuses.

In [ ]:
// Headers: Patient ID, Vital Status, Year of Death, Age at Death, Cause of Death
let vitalStatusRows = [
    // Valid inputs
    ["A000001"; "Dead"; "2017"; "60"; "Due to Cancer"] // patient died at the age of 60.0 due to cancer
    ["A000002"; "Alive"; ""; ""; ""]                   // patient is still alive
    ["A000003"; "Lost to follow-up"; ""; ""; ""]       // patient lost to follow-up

    // Invalid inputs
    ["A000004"; "Alive"; "2015"; "45"; "Due to Cancer"] // patient is alive yet has a year, age, and cause of death
    ["A000005"; "Dead"; ""; ""; ""]                     // patient dead yet has no year, age, or cause of death
]

vitalStatusRows

index,value
0,"[ A000001, Dead, 2017, 60, Due to Cancer ]"
1,"[ A000002, Alive, , , ]"
2,"[ A000003, Lost to follow-up, , , ]"
3,"[ A000004, Alive, 2015, 45, Due to Cancer ]"
4,"[ A000005, Dead, , , ]"


Each vital status row can't (and shouldn't) be read directly into a `PatientVitalStatus`. Again, each row must first be validated and only if the row is valid should it be turned into a `PatientVitalStatus`. If the row is not valid, a validation error must be returned.

Below is an example a type I'd use to represent each row in the csv... the unvalidated input for a patient vital status.

In [ ]:
type PatientVitalStatusInput = {
    PatientId: string
    Status: string
    AgeAtDeath: string
    YearOfDeath: string
    CauseOfDeath: string
}

let patientVitalStatusInputs =
    vitalStatusRows
    |> List.map (fun [patientId; vitalStatus; yearOfDeath; ageAtDeath; causeOfDeath] ->
        {
            PatientId = patientId
            Status = vitalStatus
            YearOfDeath = yearOfDeath
            AgeAtDeath = ageAtDeath
            CauseOfDeath = causeOfDeath
        }
    )

patientVitalStatusInputs

index,PatientId,Status,AgeAtDeath,YearOfDeath,CauseOfDeath
0,A000001,Dead,60,2017,Due to Cancer
1,A000002,Alive,,,
2,A000003,Lost to follow-up,,,
3,A000004,Alive,45,2015,Due to Cancer
4,A000005,Dead,,,


### Validating one input

Now that we have unvalidated inputs, we must write our validation business logic in the form of code. Each row in our csv has at least one attribute, all stringly typed. For now, we'll concentrate only one attribute, the `AgeAtDeath`. Below is code to validate that attribute, include a helper function to try to parse unsigned integers.

In [ ]:
module UnsignedInteger =
    /// Try to parse strings into unsigned integers
    ///
    ///    tryParse "64" = Some 64u
    ///    tryParse "foo" = None
    let tryParse (input: string) : uint option =
        match System.UInt32.TryParse(input) with
        | (true, num) -> Some num
        | _ -> None

module AgeAtDeath =
    // Validate that age of death is a parseable, unsigned integer.
    //
    //    validate "64" = Ok (AgeAtDath 64u)
    //    validate "foo" = Error "Not a valid age of death: foo"
    let validate (ageAtDeath: string) : Result<AgeAtDeath, string> =
        match UnsignedInteger.tryParse ageAtDeath with
        | Some age -> Ok (AgeAtDeath age)
        | _ -> Error $"Not a valid age of death: {ageAtDeath}"

// iterate through list and validate each element
[ "64"; "foo" ] |> List.map AgeAtDeath.validate

index ResultValue ErrorValue 0 age 64 <null> 1 <null> Not a valid age of death: foo

The `UnsignedInteger.tryParse` function takes in a string and returns an optional unsigned integer. If the string is a parseable unsigned integer, it will return the parsed number wrapped in the `Some` type. If the number is not a parseable unsigned integer, it will return `None`.

The `AgeAtDeath.validate` function takes in a string, attempts to parse the string into an unsigned integer, and, if successful, creates an `AgeAtDeath` with the newly parsed age.

Since '64' is parseable as an integer, it ends up becoming a valid `AgeAtDeath`. Since 'foo' is not parseable as an integer, it ends up becoming an error message.

### Composing Validations

A valid `AgeAtDeath` isn't the only valid attribute needed for a `VitalStatus`. A patient with a `VitalStatus` of `Alive` but with a valid `AgeAtDeath` is still invalid overall. Each row's attributes must be looked at together as a whole to determine if the row is a valid vital status. Below are some example validations, written in the form of `Active Expressions`, and the `PatientVitalStatus.validate` function that uses those active expressions in a match expression.

In [ ]:
let (|AliveStatus|DeadStatus|LostStatus|InvalidStatus|) input =
    match input with
    | "Alive" -> AliveStatus
    | "Dead" -> DeadStatus
    | "Lost to follow-up" -> LostStatus
    | _ -> InvalidStatus

// let (|ValidAgeAtDeath|_|)  = UnsignedInteger.tryParse >> Option.map AgeAtDeath
// let (|ValidYearOfDeath|_|) = UnsignedInteger.tryParse >> Option.map YearOfDeath
let (|ValidAgeAtDeath|_|)  = UnsignedInteger.tryParse >> Option.map AgeAtDeath
let (|ValidYearOfDeath|_|) = UnsignedInteger.tryParse >> Option.map YearOfDeath
let (|ValidCauseOfDeath|_|) causeOfDeath =
    match causeOfDeath with
    | "Due to Cancer" -> Some ``Due to Cancer``
    | "Due to Other Cause" -> Some ``Due to Other Cause``
    | "Probably Due to Cancer" -> Some ``Probably Due to Cancer``
    | "Unknown Cause" -> Some ``Unknown Cause``
    | "Unlikely Due to Cancer" -> Some ``Unlikely Due to Cancer``
    | _ -> None

module PatientVitalStatus =
    let validate (input: PatientVitalStatusInput) =
        // match input.AgeAtDeath, input.YearOfDeath, input.CauseOfDeath with
        // | ValidAgeAtDeath age, ValidYearOfDeath year, ValidCauseOfDeath cause ->
        //     match input.Status with
        //     | DeadStatus -> 
        //         Ok {
        //             PatientId = PatientId input.PatientId
        //             VitalStatus = Dead {
        //                 AgeAtDeath = age
        //                 YearOfDeath = year
        //                 CauseOfDeath = cause
        //             }
        //         }
        //     | _ -> Error $"Invalid vital status: {input}"
        // | _ ->
        //     match input.Status with
        //     | AliveStatus ->
        //         Ok {
        //             PatientId = PatientId input.PatientId
        //             VitalStatus = Alive
        //         }
        //     | LostStatus ->
        //         Ok {
        //             PatientId = PatientId input.PatientId
        //             VitalStatus = ``Lost to follow-up``
        //         }
        //     | _ -> Error $"Invalid vital status: {input}"

        match (input.Status, input.AgeAtDeath, input.YearOfDeath, input.CauseOfDeath) with
        // Valid vital status: patient is dead with a valid age at death, year of death, and cause of death
        | (DeadStatus, ValidAgeAtDeath age, ValidYearOfDeath year, ValidCauseOfDeath cause) ->
            Ok {
                PatientId = PatientId input.PatientId
                VitalStatus = Dead {
                    AgeAtDeath = age
                    YearOfDeath = year
                    CauseOfDeath = cause
                }
            }
        // Valid vital status: patient is alive with blank age at death, year of death, and cause of death
        | (AliveStatus, "", "", "") ->
            Ok {
                PatientId = PatientId input.PatientId
                VitalStatus = Alive
            }
        // Valid vital status: patient is lost to follow-up with blank age at death, year of death, and cause of death
        | (LostStatus, "", "", "") ->
            Ok {
                PatientId = PatientId input.PatientId
                VitalStatus = ``Lost to follow-up``
            }
        // Invalid vital status
        | _ ->
            Error $"Invalid vital status: {input}"

patientVitalStatusInputs
|> List.map PatientVitalStatus.validate

index ResultValue ErrorValue 0 PatientId VitalStatus { PatientId "A000001": id: A000001 } { Dead { YearOfDeath = YearOfDeath 2017u
 AgeAtDeath = AgeAtDeath 60u
 CauseOfDeath = Due to Cancer }: Item: { { YearOfDeath = YearOfDeath 2017u
 AgeAtDeath = AgeAtDeath 60u
 CauseOfDeath = Due to Cancer }: YearOfDeath: { YearOfDeath 2017u: year: 2017 }, AgeAtDeath: { AgeAtDeath 60u: age: 60 }, CauseOfDeath: Due to Cancer } } <null> 1 PatientId VitalStatus { PatientId "A000002": id: A000002 } Alive <null> 2 PatientId VitalStatus { PatientId "A000003": id: A000003 } Lost to follow-up <null> 3 <null> Invalid vital status: { PatientId = "A000004"
 Status = "Alive"
 AgeAtDeath = "45"
 YearOfDeath = "2015"
 CauseOfDeath = "Due to Cancer" } 4 <null> Invalid vital status: { PatientId = "A000005"
 Status = "Dead"
 AgeAtDeath = ""
 YearOfDeath = ""
 CauseOfDeath = "" }

## Conclusion

F# is a great language for everyday business tasks, like parsing and validating rows in a CSV. In this post, we saw a brief example of how I use F# to read in patient vital statuses and validate each status.

In my next post, I'll address how I read in XML and JSON files (F# XML Provider and `Thoth.Json.Net` respectively), as well as how I use the awesome `FsToolkit.Errorhandling` library to compose validations together.